In [1]:
from csv import reader
from random import seed, random, randrange
from tabulate import tabulate
import numpy as np
import pandas as pd

In [2]:
# Carregando um arquivo CSV
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


# Converte coluna de string em float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


# Converte coluna de string em int
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


# Encontrar os valores minimo e maximo para cada coluna do dataset
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats


# Normalizando os dados entre 0 e 1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])


# Repartindo o arquivo para aplicar k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# Percentual de acuracia
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


# Avalia algoritmo de aprendizagem usando cross validation
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    n_inputs = len(dataset[0]) - 1
    n_outputs = len(set([row[-1] for row in dataset]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)

    print_layers(network)

    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(network, train_set, test_set, n_outputs, l_rate, epsilon)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)

    print_layers(network)
    return scores


# Iniciando a rede
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network


# Calcula a ativacao do neuronio para um dado
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation


# Funcao de transferencia
def transfer(activation):
    return 1.0 / (1.0 + np.exp(-activation))


# Funcao de transferencia derivada
def transfer_derivative(output):
    return output * (1.0 - output)


# Propagando o dado para a saida da rede
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


# Retropropagacao do erro e  salvando nos neuronios
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])


# atualizar os pesos da rede
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']


# Treinando a rede por um numero fixo de epocas
def train_network(network, train, l_rate, epsilon, n_outputs):
    epoch = 0
    eqm_prev = None
    while True:
        epoch += 1
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i] - outputs[i]) ** 2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

        eqm_current = sum_error / len(train)
        if eqm_prev is None:
            eqm_prev = eqm_current
        else:
            print('>>>', abs(eqm_current - eqm_prev), ' ', epsilon)
            if abs(eqm_current - eqm_prev) <= epsilon:
                print('converged in epoch >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ', epoch)
                break
            eqm_prev = eqm_current


# Predicao com a rede treinada
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))


# Printa as camadas da rede
def print_layers(network):
    for layer in network:
        print("\n", tabulate([layer], tablefmt="plain"), "\n")

#
def back_propagation(network, train, test, n_outputs, l_rate, epsilon):
    train_network(network, train, l_rate, epsilon, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return predictions

In [3]:
seed(1)

filename = 'result_matrix_ml.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)

# convertendo coluna de classe para inteiros
str_column_to_int(dataset, len(dataset[0]) - 1)

df = pd.DataFrame(dataset)
df.iloc[:].describe()

,0,1,2,3,4
count,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000
mean,0.099023,241.931081,9.715228,7.023880,1.986791
std,0.087146,847.812442,0.354015,2.558923,0.137532
min,0.050067,0.000000,8.429426,0.007504,0.000000
25%,0.059587,4.926108,9.527781,5.487770,2.000000
50%,0.073768,12.315271,9.689797,7.239058,2.000000
75%,0.094807,19.704433,9.897037,8.641569,2.000000
max,1.377896,12879.699248,12.160811,11.869694,2.000000


In [4]:
# normalizando dados
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

df = pd.DataFrame(dataset)
df.iloc[:].describe()

,0,1,2,3,4
count,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000
mean,0.036869,0.018784,0.344591,0.591491,1.986791
std,0.065630,0.065825,0.094875,0.215721,0.137532
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.007170,0.000382,0.294356,0.461994,2.000000
50%,0.017850,0.000956,0.337776,0.609631,2.000000
75%,0.033694,0.001530,0.393315,0.727864,2.000000
max,1.000000,1.000000,1.000000,1.000000,2.000000


In [5]:
# avaliacao algoritmo
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
epsilon = 1e-07
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, epsilon, n_hidden)


 {'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095]}  {'weights': [0.4494910647887381, 0.651592972722763, 0.7887233511355132, 0.0938595867742349, 0.02834747652200631]}  {'weights': [0.8357651039198697, 0.43276706790505337, 0.762280082457942, 0.0021060533511106927, 0.4453871940548014]}  {'weights': [0.7215400323407826, 0.22876222127045265, 0.9452706955539223, 0.9014274576114836, 0.030589983033553536]}  {'weights': [0.0254458609934608, 0.5414124727934966, 0.9391491627785106, 0.38120423768821243, 0.21659939713061338]} 


 {'weights': [0.4221165755827173, 0.029040787574867943, 0.22169166627303505, 0.43788759365057206, 0.49581224138185065, 0.23308445025757263]}  {'weights': [0.2308665415409843, 0.2187810373376886, 0.4596034657377336, 0.28978161459048557, 0.021489705265908876, 0.8375779756625729]}  {'weights': [0.5564543226524334, 0.6422943629324456, 0.1859062658947177, 0.9925434121760651, 0.8599465287952899, 0.12088995980580641

>>> 1.675252679357933e-05   1e-07
>>> 1.6077269457162938e-05   1e-07
>>> 1.544567101490965e-05   1e-07
>>> 1.4854849161140618e-05   1e-07
>>> 1.4302029939353839e-05   1e-07
>>> 1.3784565881760913e-05   1e-07
>>> 1.3299947796679285e-05   1e-07
>>> 1.2845811364094878e-05   1e-07
>>> 1.241993959576887e-05   1e-07
>>> 1.2020262084676617e-05   1e-07
>>> 1.1644851836693088e-05   1e-07
>>> 1.1291920341672743e-05   1e-07
>>> 1.0959811424366556e-05   1e-07
>>> 1.0646994304704116e-05   1e-07
>>> 1.0352056208305302e-05   1e-07
>>> 1.0073694787487292e-05   1e-07
>>> 9.81071054995987e-06   1e-07
>>> 9.561999441427133e-06   1e-07
>>> 9.326545683538759e-06   1e-07
>>> 9.103414938333582e-06   1e-07
>>> 8.891747843315506e-06   1e-07
>>> 8.690753941277898e-06   1e-07
>>> 8.499706015482542e-06   1e-07
>>> 8.31793482780098e-06   1e-07
>>> 8.144824251430857e-06   1e-07
>>> 7.979806783492416e-06   1e-07
>>> 7.822359418352062e-06   1e-07
>>> 7.671999862027268e-06   1e-07
>>> 7.528283065149147e-06   1e-07
>>>

>>> 9.906605464491541e-06   1e-07
>>> 1.0174630652672392e-05   1e-07
>>> 1.045062631614517e-05   1e-07
>>> 1.0734531099738931e-05   1e-07
>>> 1.1026247056684924e-05   1e-07
>>> 1.1325637118723628e-05   1e-07
>>> 1.163252284155751e-05   1e-07
>>> 1.19466825012262e-05   1e-07
>>> 1.2267849598636069e-05   1e-07
>>> 1.2595711801130852e-05   1e-07
>>> 1.2929910309600334e-05   1e-07
>>> 1.3270039587565119e-05   1e-07
>>> 1.361564733077733e-05   1e-07
>>> 1.3966234497108168e-05   1e-07
>>> 1.4321255162522487e-05   1e-07
>>> 1.4680115935899324e-05   1e-07
>>> 1.5042174652608181e-05   1e-07
>>> 1.5406738093822966e-05   1e-07
>>> 1.5773058539973392e-05   1e-07
>>> 1.6140329070381612e-05   1e-07
>>> 1.6507677656318887e-05   1e-07
>>> 1.687416025483203e-05   1e-07
>>> 1.723875327815788e-05   1e-07
>>> 1.7600345974588816e-05   1e-07
>>> 1.7957733391769926e-05   1e-07
>>> 1.8309610692303357e-05   1e-07
>>> 1.8654569639647844e-05   1e-07
>>> 1.899109807003592e-05   1e-07
>>> 1.9317583103842587e-05   

>>> 8.233725610529846e-06   1e-07
>>> 8.216615305082185e-06   1e-07
>>> 8.199070392991316e-06   1e-07
>>> 8.181103020479634e-06   1e-07
>>> 8.162725726351336e-06   1e-07
>>> 8.143951408544348e-06   1e-07
>>> 8.124793291468951e-06   1e-07
>>> 8.1052648930411e-06   1e-07
>>> 8.085379992123394e-06   1e-07
>>> 8.065152596424888e-06   1e-07
>>> 8.044596910671518e-06   1e-07
>>> 8.023727305494707e-06   1e-07
>>> 8.002558286937522e-06   1e-07
>>> 7.981104466485596e-06   1e-07
>>> 7.959380532363523e-06   1e-07
>>> 7.937401220896312e-06   1e-07
>>> 7.915181289849284e-06   1e-07
>>> 7.892735491468739e-06   1e-07
>>> 7.870078547823726e-06   1e-07
>>> 7.847225126060217e-06   1e-07
>>> 7.824189815586018e-06   1e-07
>>> 7.800987105712788e-06   1e-07
>>> 7.777631364996353e-06   1e-07
>>> 7.754136820982069e-06   1e-07
>>> 7.730517542155028e-06   1e-07
>>> 7.706787419612705e-06   1e-07
>>> 7.682960151358768e-06   1e-07
>>> 7.6590492266021e-06   1e-07
>>> 7.63506791181829e-06   1e-07
>>> 7.6110292376117

>>> 5.929853434311939e-06   1e-07
>>> 5.931591570469512e-06   1e-07
>>> 5.933328159772426e-06   1e-07
>>> 5.93506250520532e-06   1e-07
>>> 5.936793917403829e-06   1e-07
>>> 5.938521714264275e-06   1e-07
>>> 5.940245220486563e-06   1e-07
>>> 5.941963767300967e-06   1e-07
>>> 5.943676691828881e-06   1e-07
>>> 5.945383336669954e-06   1e-07
>>> 5.9470830496462235e-06   1e-07
>>> 5.94877518315072e-06   1e-07
>>> 5.950459094001756e-06   1e-07
>>> 5.952134142661429e-06   1e-07
>>> 5.953799693116796e-06   1e-07
>>> 5.9554551124505264e-06   1e-07
>>> 5.957099770257171e-06   1e-07
>>> 5.958733038338715e-06   1e-07
>>> 5.96035429046432e-06   1e-07
>>> 5.961962901729345e-06   1e-07
>>> 5.963558248394448e-06   1e-07
>>> 5.9651397073660385e-06   1e-07
>>> 5.9667066559764e-06   1e-07
>>> 5.96825847150621e-06   1e-07
>>> 5.969794530928229e-06   1e-07
>>> 5.971314210606333e-06   1e-07
>>> 5.972816885823663e-06   1e-07
>>> 5.974301930598315e-06   1e-07
>>> 5.975768717491649e-06   1e-07
>>> 5.97721661696

...

>>> 1.1018390565220582e-07   1e-07
>>> 1.1012015319457422e-07   1e-07
>>> 1.1005643404395863e-07   1e-07
>>> 1.0999274815558149e-07   1e-07
>>> 1.0992909560067488e-07   1e-07
>>> 1.098654763202582e-07   1e-07
>>> 1.0980189036095215e-07   1e-07
>>> 1.0973833770378319e-07   1e-07
>>> 1.0967481842833176e-07   1e-07
>>> 1.0961133241349248e-07   1e-07
>>> 1.0954787980378949e-07   1e-07
>>> 1.0948446053731485e-07   1e-07
>>> 1.0942107463933048e-07   1e-07
>>> 1.0935772208186395e-07   1e-07
>>> 1.0929440292714847e-07   1e-07
>>> 1.0923111714287483e-07   1e-07
>>> 1.0916786480406979e-07   1e-07
>>> 1.0910464583169504e-07   1e-07
>>> 1.0904146027182916e-07   1e-07
>>> 1.0897830812360478e-07   1e-07
>>> 1.0891518940794702e-07   1e-07
>>> 1.0885210416529661e-07   1e-07
>>> 1.087890523186752e-07   1e-07
>>> 1.0872603393486963e-07   1e-07
>>> 1.0866304901951777e-07   1e-07
>>> 1.0860009756242812e-07   1e-07
>>> 1.0853717958138158e-07   1e-07
>>> 1.0847429506575298e-07   1e-07
>>> 1.0841144406118722

>>> 2.8766236204603567e-07   1e-07
>>> 2.800923383595782e-07   1e-07
>>> 2.7293946227072673e-07   1e-07
>>> 2.661770420481907e-07   1e-07
>>> 2.5978036163290964e-07   1e-07
>>> 2.537265135790865e-07   1e-07
>>> 2.4799424808077e-07   1e-07
>>> 2.425638356639377e-07   1e-07
>>> 2.374169427649727e-07   1e-07
>>> 2.3253651860609317e-07   1e-07
>>> 2.2790669241829885e-07   1e-07
>>> 2.2351267979297488e-07   1e-07
>>> 2.1934069757906845e-07   1e-07
>>> 2.1537788603281927e-07   1e-07
>>> 2.1161223796926747e-07   1e-07
>>> 2.0803253408026995e-07   1e-07
>>> 2.0462828351675184e-07   1e-07
>>> 2.0138966993891484e-07   1e-07
>>> 1.9830750181380752e-07   1e-07
>>> 1.953731668472838e-07   1e-07
>>> 1.9257859039812763e-07   1e-07
>>> 1.899161970330028e-07   1e-07
>>> 1.8737887530928966e-07   1e-07
>>> 1.8495994547485916e-07   1e-07
>>> 1.8265312943957588e-07   1e-07
>>> 1.8045252338358062e-07   1e-07
>>> 1.7835257241102884e-07   1e-07
>>> 1.763480470477318e-07   1e-07
>>> 1.7443402160948005e-07   1e

>>> 1.142065906657502e-07   1e-07
>>> 1.1409224179423274e-07   1e-07
>>> 1.1397792323624908e-07   1e-07
>>> 1.1386363685326594e-07   1e-07
>>> 1.1374938454697392e-07   1e-07
>>> 1.1363516817027453e-07   1e-07
>>> 1.1352098953649591e-07   1e-07
>>> 1.1340685050016588e-07   1e-07
>>> 1.1329275286669788e-07   1e-07
>>> 1.131786984554916e-07   1e-07
>>> 1.1306468898750113e-07   1e-07
>>> 1.1295072630164178e-07   1e-07
>>> 1.1283681205327342e-07   1e-07
>>> 1.1272294807567352e-07   1e-07
>>> 1.1260913602268406e-07   1e-07
>>> 1.1249537757134897e-07   1e-07
>>> 1.1238167441248153e-07   1e-07
>>> 1.1226802823852133e-07   1e-07
>>> 1.1215444064422136e-07   1e-07
>>> 1.1204091324450074e-07   1e-07
>>> 1.1192744765319441e-07   1e-07
>>> 1.1181404549779825e-07   1e-07
>>> 1.117007082350463e-07   1e-07
>>> 1.1158743746402834e-07   1e-07
>>> 1.1147423471552943e-07   1e-07
>>> 1.1136110146092032e-07   1e-07
>>> 1.1124803918675062e-07   1e-07
>>> 1.1113504936504162e-07   1e-07
>>> 1.1102213343756537e

>>> 2.3148589519534062e-07   1e-07
>>> 2.2957437783850503e-07   1e-07
>>> 2.276931083870868e-07   1e-07
>>> 2.258414491295227e-07   1e-07
>>> 2.240187791075583e-07   1e-07
>>> 2.22224493255283e-07   1e-07
>>> 2.2045800229461305e-07   1e-07
>>> 2.187187321067794e-07   1e-07
>>> 2.1700612311530837e-07   1e-07
>>> 2.153196301173197e-07   1e-07
>>> 2.1365872174250976e-07   1e-07
>>> 2.1202288002055477e-07   1e-07
>>> 2.1041159995762154e-07   1e-07
>>> 2.0882438925393272e-07   1e-07
>>> 2.0726076786271343e-07   1e-07
>>> 2.0572026760345627e-07   1e-07
>>> 2.042024319861722e-07   1e-07
>>> 2.0270681554428096e-07   1e-07
>>> 2.0123298389879574e-07   1e-07
>>> 1.9978051310173047e-07   1e-07
>>> 1.9834898961886092e-07   1e-07
>>> 1.969380098694809e-07   1e-07
>>> 1.9554717990873105e-07   1e-07
>>> 1.941761152382971e-07   1e-07
>>> 1.9282444066936674e-07   1e-07
>>> 1.9149178969574392e-07   1e-07
>>> 1.901778045884997e-07   1e-07
>>> 1.8888213600847838e-07   1e-07
>>> 1.876044428345599e-07   1e-

>>> 2.5248489372506344e-07   1e-07
>>> 2.487563696553863e-07   1e-07
>>> 2.4511603763691884e-07   1e-07
>>> 2.415609525752973e-07   1e-07
>>> 2.3808830685635458e-07   1e-07
>>> 2.3469542097481842e-07   1e-07
>>> 2.3137973517738976e-07   1e-07
>>> 2.2813880174571682e-07   1e-07
>>> 2.249702781524773e-07   1e-07
>>> 2.2187192072996298e-07   1e-07
>>> 2.188415789099304e-07   1e-07
>>> 2.15877189892362e-07   1e-07
>>> 2.129767738734585e-07   1e-07
>>> 2.101384294632587e-07   1e-07
>>> 2.073603296357104e-07   1e-07
>>> 2.046407177870536e-07   1e-07
>>> 2.0197790427320395e-07   1e-07
>>> 1.9937026300568313e-07   1e-07
>>> 1.968162283814837e-07   1e-07
>>> 1.943142923890081e-07   1e-07
>>> 1.9186300188411918e-07   1e-07
>>> 1.894609560636239e-07   1e-07
>>> 1.871068040478277e-07   1e-07
>>> 1.8479924264946327e-07   1e-07
>>> 1.8253701422859658e-07   1e-07
>>> 1.8031890476372277e-07   1e-07
>>> 1.7814374184154692e-07   1e-07
>>> 1.76010393005961e-07   1e-07
>>> 1.739177639745312e-07   1e-07
>>

In [6]:
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))

Scores: [99.50980392156863, 100.0, 100.0, 100.0, 100.0]
Mean Accuracy: 99.902%
